In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle
import logging
from tqdm import tqdm

import numpy as np
from scipy import interpolate
from astropy.io import fits
import jax.numpy as jnp

sys.path.append(os.environ['DM21CM_DIR'])
import dm21cm.physics as phys
from dm21cm.utils import load_h5_dict
from dm21cm.dm_params import DMParams
from dm21cm.evolve import get_z_edges
from dm21cm.interpolators_jax import BatchInterpolator

sys.path.append(os.environ['DH_DIR'])
import darkhistory
from darkhistory.spec.spectrum import Spectrum
from darkhistory.main import evolve as dh_evolve
from darkhistory.config import load_data as dh_load_data
print(darkhistory.__version__)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../../matplotlibrc')

1.0.0-dm21cm.0


In [3]:
tf_name = 'zf01'
abscs = load_h5_dict(os.environ['DM21CM_DIR'] + f"/data/abscissas/abscs_{tf_name}.h5")

In [13]:
# inject 1 photon
spec_N = np.zeros((500,))
i_injE = 307
spec_N[i_injE] = 1.
photeng = abscs['photE']
print(f'total energy = {np.dot(spec_N, photeng)}')

total energy = 1988413.835399729


In [14]:
rs = 38.71318413405634
xe = 0.01

## DH_run

In [11]:
dep_tf = dh_load_data(
    'dep_tf',
    verbose=1,
    prefix="/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01",
    force_reload=True
)

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01
    for propagating photons...  

 Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******


In [87]:
dh_soln = dh_evolve(
    DM_process='decay', mDM=1e10, lifetime=1e24, primary='phot_delta',
    start_rs=rs, end_rs=rs/1.01,
    coarsen_factor=12, backreaction=True,
    init_cond=(xe, phys.chi*xe, 10*phys.kB),
    use_tqdm=False, verbose=1,
    cross_check_21cmfast=True,
    debug_bath_point_injection=True,
    debug_use_tf_dt=True,
)

Loading time: 0.004 s
Initialization time: 0.026 s
rs=38.71318413405634, bath point injection
bath energy 1.988413835399729
eng per inj 10000000000.0
inj_per_Bavg 1.7007888885729e-11
inj eng 0.17007888885729
Main loop time: 0.077 s


In [88]:
f_dh = np.array([dh_soln['f']['low'][k] + dh_soln['f']['high'][k] for k in dh_soln['f']['low']])
f_dh = f_dh[:,1]

In [89]:
f_dh

array([3.33156783e-04, 5.33402993e-06, 2.68549376e-04, 3.47445583e-04,
       3.45719560e-05])

## 21 run

In [43]:
from dm21cm.evolve import evolve
import py21cmfast as p21c
from astropy.cosmology import Planck18

In [68]:
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e10,
    lifetime = 1e24,
)

return_dict = evolve(
    run_name = 'dhc_test',
    z_start = 38.2,
    z_end = 4.96266792,
    zplusone_step_factor = 1.01,
    dm_params = dm_params,
    enable_elec = False,
    tf_version = 'zf01',
    
    p21c_initial_conditions = p21c.initial_conditions(
        user_params = p21c.UserParams(
            HII_DIM = 16,
            BOX_LEN = 16 * 4, # [conformal Mpc]
            N_THREADS = 32,
        ),
        cosmo_params = p21c.CosmoParams(
            OMm = Planck18.Om0,
            OMb = Planck18.Ob0,
            POWER_INDEX = Planck18.meta['n'],
            SIGMA_8 = 1e-8, # Planck18.meta['sigma8']
            hlittle = Planck18.h,
        ),
        random_seed = 54321,
        write = True,
    ),
    
    rerun_DH = False,
    clear_cache = True,
    use_tqdm = False,
    debug_flags = ['uniform_xray'],
    debug_xray_multiplier = 1.,
    debug_astro_params = p21c.AstroParams(
        L_X = 0. # log10 value
    ),
    save_dir='../run_info',
    debug_dhc_delta_fixed=True,
    debug_use_tf_dt=True,
    debug_no_bath=True,
    debug_bath_point_injection=True,
    debug_break_after_z=37.,
)

INFO:root:Using 21cmFAST version 0.1.dev1581+g40959ca.d20230929
INFO:root:Cache dir: /n/holyscratch01/iaifi_lab/yitians/21cmFAST-cache/dhc_test
INFO:root:TransferFunctionWrapper: Loaded photon transfer functions.
INFO:root:DarkHistoryWrapper: Found existing DarkHistory initial conditions.


get_init_cond 61.559512985154036 0.001043024536618191
This is commented out: dspec_dzp = - dxe_dzp * T / (1+x_e); need to put it back in
box_ct = 0
dzp = -9.587479e-02
zp = 3.810032e+01
dt_dzp = -8.572903e+13
dadia_dzp * dzp = -1.478124e-01
dspec_dzp * dzp = 0.000000e+00
dcomp_dzp * dzp = 7.661585e-03
dxheat_dzp * dzp = 0.000000e+00
input heating = 0.000000e+00
dxion_source = 0.000000e+00
dxion_sink = 7.227056e-08
T = 3.014094e+01
alpha_A(T) = 1.790334e-11
global_params.CLUMPING_FACTOR = 1.000000e+00
x_e = 2.078851e-04
f_H = 9.249617e-01
prefactor_1 = 1.228640e-02
curr_delNL0 = -1.830332e-06
growth_factor_zp = 3.256493e-02
input ionization = 0.000000e+00

N_b0 = 2.055340e-07
f_H = 9.249617e-01

dt 33356785139759.77 33677476183046.547
before 61.55951 0.0010430245
input_heating 2.728844e-06
This is commented out: dspec_dzp = - dxe_dzp * T / (1+x_e); need to put it back in
box_ct = 0
dzp = -3.871307e-01
zp = 3.771318e+01
dt_dzp = -8.788830e+13
dadia_dzp * dzp = -1.231187e+00
dspec_dzp * d

dt 33860104410955.453 34050802162151.98
bath energy 1.9884138
eng per inj 10000000000.000002
inj_per_Bavg 1.7007892e-11
inj eng 0.17007891622844443
before 60.416275 0.0010382861
input_heating 0.2844925
This is commented out: dspec_dzp = - dxe_dzp * T / (1+x_e); need to put it back in
box_ct = 0
dzp = -3.833008e-01
zp = 3.732988e+01
dt_dzp = -9.010197e+13
dadia_dzp * dzp = -1.208332e+00
dspec_dzp * dzp = 0.000000e+00
dcomp_dzp * dzp = 8.560505e-02
dxheat_dzp * dzp = 0.000000e+00
input heating = 2.844925e-01
dxion_source = 0.000000e+00
dxion_sink = 4.679031e-06
T = 6.041629e+01
alpha_A(T) = 1.173891e-11
global_params.CLUMPING_FACTOR = 1.000000e+00
x_e = 1.038286e-03
f_H = 9.249617e-01
prefactor_1 = 1.157434e-02
curr_delNL0 = -1.794200e-06
growth_factor_zp = 3.322074e-02
input ionization = 4.803791e-06

N_b0 = 2.055340e-07
f_H = 9.249617e-01

after 59.57806 0.0010384109
----- DM21CM -----
z 37.71318419519382
[3.64213308e-09 5.58752945e-11 3.10500895e-09 2.09205219e-09
 5.34460710e-10 2.08

In [ ]:
# DM21CM:
# WARNING:root:Point injecting bath at z=37.713184
# dt 33860104410955.453 34050802162151.98
# bath energy 1.9884138
# eng per inj 10000000000.000002
# inj_per_Bavg 1.7007892e-11
# inj eng 0.17007891622844443

In [ ]:
# DarkHistory
# rs=38.71318413405634, bath point injection
# bath energy 1.988413835399729
# eng per inj 10000000000.0
# inj_per_Bavg 1.7007888885729e-11
# inj eng 0.17007888885729

In [80]:
run_name = 'dhc_test'
soln = np.load(f"../run_info/{run_name}_records.npy", allow_pickle=True).item()

In [81]:
soln['z']

array([38.10031604, 37.7131842 , 37.32988534, 36.95038153, 36.57463517])

In [85]:
np.array([soln['f_ion'][2], soln['f_exc'][2], soln['f_heat'][2]])

array([0.00038659, 0.00032374, 0.00021774])

In [90]:
np.array([f_dh[0] + f_dh[1], f_dh[2], f_dh[3]])

array([0.00033849, 0.00026855, 0.00034745])

## compare

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL+1)

fig, ax = plt.subplots()

ax.plot(f_dh[:4], label='DarkHistory', color='k')
ax.plot(f_dm21cm[:4], label='dm21cm', color='r', ls=':')
ax.set_xticks(np.arange(4))
ax.set_xticklabels([s.decode('utf-8') for s in dmcm_dep_tf.abscs['out'][:4]])
ax.set(yscale='log')
#ax.set(ylim=(1e-7,1e-4))
ax.legend();

In [ ]:
f_dm21cm[:4] / f_dh[:4]

array([1.        , 0.99999994, 1.        , 0.99999993])

## A. HED pipeline

In [110]:
# .npy
data_dir = f"{os.environ['DM21CM_DATA_DIR']}/tf/zf01/phot"
hed_tfgv = np.load(f'{data_dir}/hed_tf_rxneo.npy')
hed_tfgv[8, 3, 6, 429]

array([11.0582994 ,  8.78602481,  3.48876916, -0.0763134 ])

In [111]:
# dm21cm tf
tf = np.load(f"{os.environ['DM21CM_DATA_DIR']}/tf/zf01/phot/hed_tf_rxneo.npy")
tf[8, 3, 6, 429]

array([11.0582994 ,  8.78602481,  3.48876916, -0.0763134 ])

In [134]:
# dhtf
dhtf = pickle.load(open(f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_zf01/highengdep_interp.raw", 'rb'))
dhtf.grid_vals[0][3, 0, 8, 429] * (abscs['dlnz'] / phys.hubble(abscs['rs'][-2]))

array([11.0582994 ,  8.78602481,  3.48876916, -0.0763134 ])

In [135]:
dhtf.get_val(0.01, 0.9, abscs['rs'][-2])[429] * (abscs['dlnz'] / phys.hubble(abscs['rs'][-2]))

array([11.0582994 ,  8.78602481,  3.48876916, -0.0763134 ])

correct!!